In [ ]:
import os, gspread, openpyxl, fnmatch, requests, time, datetime, re, pickle
import pandas as pd
import numpy as np
from datetime import date

from selenium import webdriver
from selenium.webdriver import Firefox
from selenium.webdriver import DesiredCapabilities
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
from time import sleep
from helium import *
from bs4 import BeautifulSoup


#Gmail API utils
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
#for encoding/decoding messages in base64
from base64 import urlsafe_b64decode, urlsafe_b64encode

This code imports various libraries and modules needed for a Python script that automates a reporting process. The script uses gspread, openpyxl, and pandas for working with spreadsheets, Selenium and Helium for web scraping, the Google API for accessing email data and Google Sheets, and BeautifulSoup for parsing HTML content. 

In [ ]:
# selenium driver launch
profile_path = r'<File Path>'
options = Options()
options.profile = profile_path
service = Service('geckodriver.exe')
driver = start_firefox('<URL>',options=options, headless=False)



Selenium driver launch:
This section sets up and launches a Selenium WebDriver for Firefox browser automation with a specified profile path and options.

In [ ]:
advertising_cred = r'<File Path to credientials.json>'
advertising_user = r'<File Path to authorized_user.json>'

# Request Gmail all access (permission to read/send/receive emails, manage the inbox, and more)
SCOPES = ['https://mail.google.com/']

File paths for credentials and authorized users:
This part defines file paths for different Google API credentials and authorized users.

In [ ]:
# Google Sheet Authenticate
gc = gspread.oauth(
    credentials_filename = advertising_cred,
    authorized_user_filename = advertising_user
)

Google Sheets authentication:
This part initializes a gspread client using the provided credentials.

In [ ]:
# Gmail Authentication

def gmail_authenticate():
    creds = None
    if os.path.exists("token.pickle"):
        with open("token.pickle", "rb") as token:
            creds = pickle.load(token)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(r'C:\Users\csakp\AppData\Roaming\gspread\credentials-esb-m.json', SCOPES)
            creds = flow.run_local_server(port=0)
        with open("token.pickle", "wb") as token:
            pickle.dump(creds, token)
    return build('gmail', 'v1', credentials=creds)

service = gmail_authenticate()

Gmail authentication:
This section defines a function, gmail_authenticate(), that authenticates the user with Gmail using a token.pickle file and the provided credentials. It returns a Gmail API service object.

In [ ]:
# log into platform
def websiteDownload():
    print('Logging into <"Platform">...')
    
    write([Username], into='Username')
    write([Password], into='Password')
    click('Log in')
    time.sleep(3)

    # Search for "Ready for download" and zoom out to find all reports
    write('Ready for download', into=TextField('Filter Reports'))
    
    driver.set_context("chrome")
    win = driver.find_element_by_tag_name("html")
    for _ in range(5):
        win.send_keys(Keys.CONTROL + "-")
    driver.set_context("content")
    time.sleep(3)
    
    # Download and save reports to folder
    report_links = driver.find_elements_by_link_text('Ready for download')
    
    print('Downloading <Platform> Reports')
    for i, link in enumerate(report_links):
        switch_to('<Platform> - Report Builder')
        click(link)
        print(f'{i}. ', end=' ')
        time.sleep(2)
        
    print('\nFinished downloading [Platform] reports')
    switch_to('<Platform> - Report Builder')

This function logs into the platform with Selenium, filters the reports, zooms out in browser to find all reports, and downloads them by clicking Ready for download button.

In [ ]:
def download_LSA():
    # Download LSA from MCC with Selenium
    go_to('<Google LSA Ads MCC URL>')
    time.sleep(5)
    
    # Select custom date range in drop down
    click(S("//*[@class=\"DPvwYc\"]"))
    time.sleep(0.5)
    click("Custom")
    time.sleep(0.5)
    
    # Set start date to previous month's first day
    x = datetime.datetime.now()
    start_date = x.replace(day=1) - datetime.timedelta(days=1)
    start_date_str = start_date.strftime('%m/%d/%Y')
    write(start_date_str, into=TextField('Start date'))
    
    # Set end date to previous month's last day
    end_date = x.replace(day=1) - datetime.timedelta(days=1)
    end_date_str = end_date.strftime('%m/%d/%Y')
    write(end_date_str, into=TextField('End date'))
    
    # Click apply
    click(S("//*[@jsname=\"k2hvje\"]"))
    time.sleep(4.5)
    
    # Generate report
    switch_to('MCC Reports')
    click(S("//*[@jsname=\"hRZeKc\"]"))
    time.sleep(.5)
    
    # Download report
    click(Button("Download"))
    time.sleep(2)
    
    # If download button doesn't work, try again
    if Text('Please select a report to download.').exists():
        click(Button("Download"))
        time.sleep(2)
        
    driver.close()
    print('LSA Downloaded', '\n')


This function uses Selenium to download a Local Service Ads report from the Google LSA Ads MCC website. It selects a custom date range and downloads the report, retrying if necessary. The downloaded file is then saved to a specified location.

In [ ]:
def downloadMW():
    print('Getting MW Report URL')
    # Get the latest email with MW thread
    result = service.users().messages().list(userId='me', q="<Report Name>").execute()
    result = result['messages'][0]

    # Parse email extracting 'data' section for full URL
    msg = service.users().messages().get(
        userId='me',
        id=result['id'],
        format='full').execute()

    # Decode data into URL string
    nText = str(urlsafe_b64decode(msg['payload']['body']['data']).decode())
    dlUrl = re.search("(?P<url>https?://[^\s]+)", nText).group("url")

    r = requests.get(dlUrl, allow_redirects=True)
    open(r'PATH/TO/DESTINATION/<Report>.csv', 'wb').write(r.content)

    print('Downloaded Report')
    print('Finished Downloading Reports')

This function downloads the latest reports from an email, extracts the download URL from the email, and downloads the report file to a specified location on the local machine.

In [ ]:
# Upload Call Tracking Report To GSheet
def msreport():
    path = os.chdir(r'PATH/TO/DESTINATION/<Reports Folder>')
    mfile = next((name for name in os.listdir(path) if fnmatch.fnmatch(name, '<Report Name Prefix>')), '')

    print(f'Found {mfile}')

    fh = mfile
    df = pd.read_excel(fh, usecols="A:G")
    print('Done reading xlsx file...')

    sh = gc.open('<Google Sheets of Report> ')
    wks = sh.worksheet('<Worksheet Name>')
    print('Converting formatting...')
    
    
    df['Lead'] = df['Lead'].astype(bool).astype(str)
    df['Call Date'] = df['Call Date'].dt.strftime('%m/%d/%Y')
    df = df.fillna('').drop(df.tail(1).index)
    print(f'New data date: {df.iloc[-1][0]}')
    
    pasteOn = wks.acell('A5').value
    print(wks.update(f"{pasteOn}:G",df.values.tolist(),value_input_option="user_entered"))
    print(f'Done updating {sh.title}')

This code reads an Excel file, converts data types, fills NaN values, and updates a Google Sheet with the data. It first searches for the Excel file in a directory, then reads it into a pandas dataframe. After converting and filling the data, it updates a specific worksheet in a Google Sheet with the new data.

In [ ]:
#Create Metrics and Domains Dictionaries
metricsDict = {}
domainsDict = {}

def create_metrics_dict():
    print('Creating Metrics Dict')
    path = os.chdir(r'PATH/TO/DESTINATION/<Reports Folder>')
    sh = gc.open('Daily Reporting Info')
    wks = sh.worksheet('APA')
    
    apa_dict = {}
    values_list = wks.get_all_values()

    for row in values_list[1:]:
        if row[1] == 'Active':
            apa_dict[row[0]] = {
                'Gsheet': row[3],
                'Worksheet': row[5],
                'URL': row[6],
                'Csv': row[7]
            }
    
    print('APA Dictionary Created')
    

def create_domains_dict():
    print('Creating Domains Dict')
    path = os.chdir(r'PATH/TO/DESTINATION/<Reports Folder>')
    sh = gc.open('Daily Reporting Info')
    wks = sh.worksheet('Domains')
    
    domains_dict = {}
    values_list = wks.get_all_values()
    
    for row in values_list[1:]:
        if row[1] == 'Active':
            domains_dict[row[0]] = {
                'Gsheet': row[3],
                'Worksheet': row[5],
                'URL': row[6],
                'Csv': row[7],
                'lastCol': row[9]
            }

    print('Domains Dictionary Created')

The code defines two functions, create_metrics_dict() and create_domains_dict(), that create dictionaries containing information about metrics and domains, respectively. The functions read data from a Google Sheet named "Daily Reporting Info" and extract the necessary information based on certain criteria, such as only considering rows with an "Active" status. The resulting dictionaries are used later in the program to retrieve specific information about metrics and domains.

In [ ]:
def update_apa_reports():
    count = 1

    for account in apaDict:
        print(f'Starting {account}\'s report {count}/{len(apaDict)}')
        mfile = apaDict[account]['Csv']
        print(f'Opening {mfile}')
        df = pd.read_csv(mfile, header=4)

        sh = gc.open_by_key(apaDict[account]['Gsheet'])
        wks = sh.get_worksheet_by_id(int(apaDict[account]['Worksheet']))
        
        rl = wks.find("Rows Left")
        rowsleft = wks.cell(rl.row+1, rl.col).value
        print(f'Rows left {rowsleft}')
        oldRowcount = wks.row_count
        print(f'Old row count {oldRowcount}')
        
        print(f'New data date: {df.iloc[-1][0]}')
        df['Date'] = df['Date'].astype(str)
        df['Order ID'] = df['Order ID'].astype(str)
        df = df.fillna('')
        df.dropna(how='all',inplace=True)

        cell = wks.find("Paste On")
        pasteOn = wks.cell(cell.row+1, cell.col).value
        print(f'Pasting on {pasteOn}')

        print(wks.update(f"{pasteOn}:L",df.values.tolist(),value_input_option="user_entered"))
        print(f'Done updating {sh.title}' + '\n')
        count += 1
        time.sleep(20)

    print(f'Finished Updating APA Reports')
    
    
def update_domains_reports():
    count = 1
    today = date.today()
    todayStr = date.isoformat(today)

    for account in domainsDict:
        print(f'Starting {account}\'s report {count}/{len(domainsDict)}')
        mfile = domainsDict[account]['Csv']
        
        print(f'Opening {mfile}')
        df = pd.read_csv(mfile, header=4)
        
        sh = gc.open_by_key(domainsDict[account]['Gsheet'])
        wks = sh.get_worksheet_by_id(int(domainsDict[account]['Worksheet']))
        oldRowcount = wks.row_count
        print(f'Old row count {oldRowcount}')
        
        
        print(f'New data date: {df.iloc[-1][0]}')
        df[df.columns[0]] = df[df.columns[0]].astype(str)
        df['Order ID'] = df['Order ID'].astype(str)
        df = df.fillna('')
        df.dropna(how='all', inplace=True)

        cell = wks.find("Paste On")
        pasteOn = wks.cell(cell.row+1, cell.col).value
        print(f'Pasting on {pasteOn}')
        print(f"Last col is {domainsDict[account]['lastCol']}")
        
        print(wks.update(f"{pasteOn}:{domainsDict[account]['lastCol']}", df.values.tolist(), value_input_option="user_entered"))
        
        lu_cell = wks.find("Last Updated")
        wks.update_cell(lu_cell.row+1, lu_cell.col, todayStr)
        
        print(f'Last Updated Date changed to {todayStr}')
        print(f'Done updating {sh.title}' + '\n')
        count += 1
        time.sleep(20)

    print(f'Finished Updating Domains Reports')

The update_apa_reports() function loops through each account in the apaDict dictionary, reads in a CSV file, and pastes the data into a specific worksheet within a Google Sheet, with the paste location defined by a cell labeled "Paste On" within the worksheet. It also updates the cell containing the number of rows left and the "New data date" value within the console output.

Similarly, the update_domains_reports() function loops through each account in the domainsDict dictionary, reads in a CSV file, and pastes the data into a specific worksheet within a Google Sheet. In addition to updating the "Paste On" cell and the "New data date" value within the console output, this function also updates the "Last Updated" date for the worksheet.

In [ ]:
def lsa_report():
    path = os.chdir(r'PATH/TO/DESTINATION/<Reports Folder>')

    # Find the latest LSA account report file
    mfile = next((name for name in os.listdir(path) if fnmatch.fnmatch(name, 'LSA_account*')), '')
    print(f'Found {mfile}')

    # Read the CSV file into a DataFrame
    df = pd.read_csv(os.path.join(path, mfile))
    print('Opening CSV file...')

    # Open the target Google Sheet and select the worksheet to update
    sh = gc.open('LSA Billing 2022')
    wks = sh.worksheet('LSA Paste')
    print('Converting formatting...')

    # Fill any missing values with an empty string
    df = df.fillna('')

    # Find the cell to start pasting data on
    pasteOn = wks.acell('A5').value

    # Update the worksheet with the data from the CSV file
    print(wks.update(f"{pasteOn}:Q", df.values.tolist(), value_input_option="user_entered"))
    
    # Update the Last Updated cell with the current date
    today = date.today()
    todayStr = date.isoformat(today)
    LUpaste = wks.find("Last Updated").offset(1, 0)
    LUpaste.value = todayStr
    wks.update_cell(LUpaste.row, LUpaste.col, todayStr)
    print(f'Last Updated Date changed to {todayStr}')
    print(f'Done updating {sh.title}')

    

This code automates the updating of a Google Sheet with data from a CSV file containing LSA account report. It looks for the latest LSA account report file in the specified folder, reads it into a DataFrame, and updates the target Google Sheet with the data from the CSV file. The code fills any missing values with an empty string and updates the "Last Updated" cell with the current date.

In [ ]:
def clearFolder():
    path = os.chdir(r'PATH/TO/DESTINATION/<Reports Folder>')
    for name in os.listdir(path):
            os.remove(name)
    print('Folder Cleared')


The code clears all files in a specified folder by iterating through the files and deleting them using the os.remove() method. It then prints "Folder Cleared".